In [4]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [6]:
subject = 'Maranhão - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [7]:
data = pd.read_csv('2003_01_model_input_MA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Air Relative Humidity (%),Wind Gust (m/s),Maranhão - Desemprego,Maranhão - IDH,anhã-03o - Produção de Cimento (t),Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,Maranhão - Consumo de Cimento (t)
0,2003,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,31.748725,1005.764849,...,80.102881,2.345134,8.213662,0.635570,293.705283,4.315322e+07,2.636001e+06,6.034057,3.872495e+07,470.726000
1,2004,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,29.321112,1005.804145,...,79.413352,2.286203,8.145546,0.640694,308.130986,4.381448e+07,2.646202e+06,6.058375,3.895418e+07,531.005000
2,2005,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,28.222592,1005.799555,...,79.717130,2.409519,8.122342,0.645312,308.093212,4.443858e+07,2.668922e+06,6.082069,3.916195e+07,579.458000
3,2006,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,26.802610,1005.575739,...,80.027395,2.477613,8.038324,0.649437,322.991255,4.501666e+07,2.698856e+06,6.102754,3.934191e+07,693.939000
4,2007,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,25.792300,1005.161290,...,80.726271,2.218828,8.007661,0.653077,339.260912,4.550078e+07,2.671721e+06,6.110757,3.943349e+07,816.847000
5,2008,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,25.391999,1004.988378,...,80.957818,1.959990,7.978325,0.656244,352.164879,4.598085e+07,2.691136e+06,6.129297,3.956333e+07,963.205000
6,2009,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,28.525452,1004.900606,...,81.606682,1.848325,8.007891,0.658950,363.707506,4.630053e+07,2.690232e+06,6.126304,3.956853e+07,1123.586000
7,2010,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,31.850842,1004.891956,...,83.294747,1.861175,7.971707,0.661204,424.052860,4.751040e+07,2.700684e+06,6.153755,3.988864e+07,1378.741000
8,2011,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,33.204591,1005.053475,...,84.184292,1.989252,7.961524,0.663017,558.996171,4.866181e+07,2.701549e+06,6.182820,4.018069e+07,1282.016000
9,2012,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,31.596977,1005.214783,...,83.319039,1.993164,10.302202,0.664173,688.169638,4.972690e+07,2.696497e+06,6.208137,4.043865e+07,1528.770000


In [8]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - Desemprego,Maranhão - IDH,anhã-03o - Produção de Cimento (t),Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado
0,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,31.748725,1005.764849,16470.339351,...,23.091064,80.102881,2.345134,8.213662,0.635570,293.705283,4.315322e+07,2.636001e+06,6.034057,3.872495e+07
1,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,29.321112,1005.804145,16999.510827,...,23.050946,79.413352,2.286203,8.145546,0.640694,308.130986,4.381448e+07,2.646202e+06,6.058375,3.895418e+07
2,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,28.222592,1005.799555,16854.168223,...,23.068373,79.717130,2.409519,8.122342,0.645312,308.093212,4.443858e+07,2.668922e+06,6.082069,3.916195e+07
3,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,26.802610,1005.575739,14247.363124,...,23.042744,80.027395,2.477613,8.038324,0.649437,322.991255,4.501666e+07,2.698856e+06,6.102754,3.934191e+07
4,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,25.792300,1005.161290,9268.561620,...,23.059765,80.726271,2.218828,8.007661,0.653077,339.260912,4.550078e+07,2.671721e+06,6.110757,3.943349e+07
5,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,25.391999,1004.988378,5429.125013,...,23.175432,80.957818,1.959990,7.978325,0.656244,352.164879,4.598085e+07,2.691136e+06,6.129297,3.956333e+07
6,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,28.525452,1004.900606,3611.878029,...,23.414376,81.606682,1.848325,8.007891,0.658950,363.707506,4.630053e+07,2.690232e+06,6.126304,3.956853e+07
7,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,31.850842,1004.891956,2589.577614,...,23.782318,83.294747,1.861175,7.971707,0.661204,424.052860,4.751040e+07,2.700684e+06,6.153755,3.988864e+07
8,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,33.204591,1005.053475,2032.576805,...,24.015643,84.184292,1.989252,7.961524,0.663017,558.996171,4.866181e+07,2.701549e+06,6.182820,4.018069e+07
9,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,31.596977,1005.214783,1722.082838,...,23.760545,83.319039,1.993164,10.302202,0.664173,688.169638,4.972690e+07,2.696497e+06,6.208137,4.043865e+07


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      531.005000
1      579.458000
2      693.939000
3      816.847000
4      963.205000
5     1123.586000
6     1378.741000
7     1282.016000
8     1528.770000
9     1563.125000
10    1303.411571
11    1191.509657
12    1100.217771
13     971.921000
14     750.012000
15     774.045000
16    1038.249000
17    1059.129000
18    1083.999000
19            NaN
Name: Maranhão - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - Desemprego,Maranhão - IDH,anhã-03o - Produção de Cimento (t),Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado
0,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,1.360097,0.826935,1.753765,...,-0.896763,-1.025784,1.388215,-0.828876,-2.014052,-1.096791,-1.621261,-0.224145,-1.998287,-1.870736
1,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,0.227870,0.940289,1.840978,...,-1.043951,-1.560723,1.136232,-0.862473,-1.605840,-0.997896,-1.414909,-0.023082,-1.612382,-1.530907
2,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-0.284475,0.927049,1.817024,...,-0.980013,-1.325051,1.663515,-0.873918,-1.237891,-0.998155,-1.220153,0.424735,-1.236379,-1.222893
3,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-0.946748,0.281431,1.387395,...,-1.074043,-1.084346,1.954674,-0.915359,-0.909344,-0.896021,-1.039757,1.014736,-0.908125,-0.956105
4,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-1.417951,-0.914084,0.566837,...,-1.011595,-0.542155,0.848147,-0.930483,-0.619336,-0.784484,-0.888683,0.479902,-0.781118,-0.820337
5,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-1.604650,-1.412865,-0.065943,...,-0.587224,-0.362520,-0.258606,-0.944952,-0.367005,-0.696021,-0.738873,0.862576,-0.486904,-0.627858
6,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.143222,-1.666052,-0.365444,...,0.289437,0.140872,-0.736070,-0.930370,-0.151490,-0.616890,-0.639114,0.844749,-0.534398,-0.620145
7,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,1.407724,-1.691005,-0.533930,...,1.639376,1.450480,-0.681122,-0.948217,0.028073,-0.203191,-0.261563,1.050767,-0.098789,-0.145584
8,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,2.039106,-1.225088,-0.625729,...,2.495424,2.140593,-0.133486,-0.953239,0.172546,0.721915,0.097744,1.067819,0.362454,0.287365
9,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,1.289322,-0.759780,-0.676902,...,1.559496,1.469326,-0.116761,0.201265,0.264666,1.607466,0.430115,0.968230,0.764210,0.669794


In [11]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      531.005000
1      579.458000
2      693.939000
3      816.847000
4      963.205000
5     1123.586000
6     1378.741000
7     1282.016000
8     1528.770000
9     1563.125000
10    1303.411571
11    1191.509657
12    1100.217771
13     971.921000
14     750.012000
15     774.045000
16    1038.249000
Name: Maranhão - Consumo de Cimento (t), dtype: float64

In [12]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - Desemprego,Maranhão - IDH,anhã-03o - Produção de Cimento (t),Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado
17,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,-0.547938,0.236151,-0.684249,...,0.061719,0.739102,-1.191,0.997695,1.039369,-0.622646,0.843673,-0.833746,0.580394,0.644008


In [13]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    1059.129
Name: Maranhão - Consumo de Cimento (t), dtype: float64

In [14]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [15]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [17]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[4050142928, 1824846574, 2697447439, 1629328878, 3822967771, 4048494483, 3393546244, 2356540453, 1125047848, 1829067920, 1496711573, 483318173, 1424199502, 1964003758, 3902032393, 2390448154, 3861290161, 1391815545, 456466209, 2769436370, 3200274395, 239182746, 3598936393, 334096169, 2044924998, 4090766470, 1856177042, 97777450, 335232562, 2514794668, 3676745999, 4169828901, 1197900533, 584992769, 3807425190, 1751038051, 69097638, 1019639777, 1098251176, 419883941, 2018333716, 762991644, 2630515062, 2593988463, 280972341, 3929614610, 2043100063, 1787901526, 2718778169, 1700590240, 266768268, 665310899, 731950010, 1208245715, 206333211, 4248980722, 4141014093, 95474706, 2589428877, 546643874, 3769301969, 2506835281, 638917253, 2080150559, 234683225, 269459360, 3096561375, 4025821905, 3253445638, 1661222749, 4282878609, 149993153, 213150380, 4198868191, 805832506, 4266967669, 3383822657, 1144651612, 3702209214, 637676214, 3077437760, 3771026793, 1514162799, 113858432, 3293234379, 1380911

2023-10-04 14:02:06.531968: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-04 14:02:06.566959: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 19396.08984375


Step: 24 ___________________________________________
val_loss: 30753.390625


Step: 25 ___________________________________________
val_loss: 29989.3671875


Step: 26 ___________________________________________
val_loss: 17611.609375


Step: 27 ___________________________________________
val_loss: 62529.76953125


Step: 28 ___________________________________________


2023-10-04 14:03:56.661202: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-04 14:03:56.699202: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-04 14:03:57.000366: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-04 14:03:57.026473: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-04 14:03:57.061130: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 60379.40234375


Step: 29 ___________________________________________
val_loss: 17094.935546875


Step: 30 ___________________________________________
val_loss: 26253.556640625


Step: 31 ___________________________________________
val_loss: 25307.3671875


Step: 32 ___________________________________________
val_loss: 45946.234375


Step: 33 ___________________________________________
val_loss: 17052.271484375


Step: 34 ___________________________________________
val_loss: 17054.857421875


Step: 35 ___________________________________________
val_loss: 17060.310546875


Step: 36 ___________________________________________
val_loss: 34331.88671875


Step: 37 ___________________________________________
val_loss: 40425.2578125


Step: 38 ___________________________________________
val_loss: 17162.5078125


Step: 39 ___________________________________________
val_loss: 28615.8125


Step: 40 ___________________________________________
val_loss: 45121.25


Step: 41 ______________

...

val_loss: 17116.708984375


Step: 55 ___________________________________________
val_loss: 17054.521484375


Step: 56 ___________________________________________
val_loss: 18075.0546875


Step: 57 ___________________________________________
val_loss: 35740.48046875


Step: 58 ___________________________________________
val_loss: 20729.1484375


Step: 59 ___________________________________________
val_loss: 19934.591796875


Step: 60 ___________________________________________
val_loss: 17098.6796875


Step: 61 ___________________________________________
val_loss: 30819.185546875


Step: 62 ___________________________________________
val_loss: 63088.04296875


Step: 63 ___________________________________________
val_loss: 46806.35546875


Step: 64 ___________________________________________
val_loss: 79883.0859375


Step: 65 ___________________________________________
val_loss: 17607.244140625


Step: 66 ___________________________________________
val_loss: 17063.0


Step: 67 __________

2023-10-04 14:22:13.751220: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-04 14:22:13.780815: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-04 14:22:14.071755: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-04 14:22:14.100600: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 19254.962890625


Step: 94 ___________________________________________
val_loss: 17520.263671875


Step: 95 ___________________________________________
val_loss: 24094.2421875


Step: 96 ___________________________________________
val_loss: 26304.119140625


Step: 97 ___________________________________________


2023-10-04 14:23:04.520891: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 94119.3984375


Step: 98 ___________________________________________
val_loss: 17588.71875


Step: 99 ___________________________________________
val_loss: 19029.427734375


final_seed: 3929614610


In [18]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 456ms/step - loss: 1164363.0000 - val_loss: 694540.8125
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 1113559.0000 - val_loss: 85003.3906
Epoch 3/10000
1/1 [==============================] - 0s 13ms/step - loss: 1251440.1250 - val_loss: 158143.3594
Epoch 4/10000
1/1 [==============================] - 0s 13ms/step - loss: 1030563.7500 - val_loss: 519406.6562
Epoch 5/10000
1/1 [==============================] - 0s 13ms/step - loss: 1054258.8750 - val_loss: 403339.3438
Epoch 6/10000
1/1 [==============================] - 0s 13ms/step - loss: 1063467.8750 - val_loss: 368689.7500
Epoch 7/10000
1/1 [==============================] - 0s 13ms/step - loss: 1024111.4375 - val_loss: 398978.7500
Epoch 8/10000
1/1 [==============================] - 0s 13ms/step - loss: 1008553.7500 - val_loss: 359500.5938
Epoch 9/10000
1/1 [==============================] - 0s 13ms/step - loss: 982443.5000 - val_loss: 378055.5312
Ep

Epoch 75/10000
1/1 [==============================] - 0s 13ms/step - loss: 232918.7344 - val_loss: 141041.5781
Epoch 76/10000
1/1 [==============================] - 0s 13ms/step - loss: 229598.9844 - val_loss: 102865.9375
Epoch 77/10000
1/1 [==============================] - 0s 13ms/step - loss: 234994.9219 - val_loss: 106679.5000
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 257456.2031 - val_loss: 65492.7305
Epoch 79/10000
1/1 [==============================] - 0s 13ms/step - loss: 237568.5781 - val_loss: 141147.9531
Epoch 80/10000
1/1 [==============================] - 0s 13ms/step - loss: 228130.9219 - val_loss: 112983.8125
Epoch 81/10000
1/1 [==============================] - 0s 13ms/step - loss: 220208.1250 - val_loss: 98072.8359
Epoch 82/10000
1/1 [==============================] - 0s 13ms/step - loss: 221069.5781 - val_loss: 116290.2891
Epoch 83/10000
1/1 [==============================] - 0s 13ms/step - loss: 249617.8281 - val_loss: 86392.1328
Epoc

1/1 [==============================] - 0s 15ms/step - loss: 108771.1484 - val_loss: 11559.5625
Epoch 150/10000
1/1 [==============================] - 0s 14ms/step - loss: 81291.6875 - val_loss: 10107.2109
Epoch 151/10000
1/1 [==============================] - 0s 13ms/step - loss: 80664.2891 - val_loss: 13762.0361
Epoch 152/10000
1/1 [==============================] - 0s 13ms/step - loss: 79724.9844 - val_loss: 10036.1641
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 75826.7188 - val_loss: 30823.9082
Epoch 154/10000
1/1 [==============================] - 0s 13ms/step - loss: 70673.6484 - val_loss: 27944.6035
Epoch 155/10000
1/1 [==============================] - 0s 13ms/step - loss: 72876.0078 - val_loss: 17600.0801
Epoch 156/10000
1/1 [==============================] - 0s 13ms/step - loss: 79601.1094 - val_loss: 26291.8379
Epoch 157/10000
1/1 [==============================] - 0s 13ms/step - loss: 60627.7812 - val_loss: 26818.1465
Epoch 158/10000
1/1 [====

1/1 [==============================] - 0s 13ms/step - loss: 51106.9531 - val_loss: 23271.7500
Epoch 224/10000
1/1 [==============================] - 0s 13ms/step - loss: 56777.1836 - val_loss: 17092.7480
Epoch 225/10000
1/1 [==============================] - 0s 13ms/step - loss: 36765.1914 - val_loss: 17071.3594
Epoch 226/10000
1/1 [==============================] - 0s 13ms/step - loss: 41432.1445 - val_loss: 17576.8398
Epoch 227/10000
1/1 [==============================] - 0s 13ms/step - loss: 44028.9688 - val_loss: 17293.3848
Epoch 228/10000
1/1 [==============================] - 0s 13ms/step - loss: 42980.3750 - val_loss: 17250.9277
Epoch 229/10000
1/1 [==============================] - 0s 13ms/step - loss: 36940.5781 - val_loss: 17057.5996
Epoch 230/10000
1/1 [==============================] - 0s 13ms/step - loss: 41577.7188 - val_loss: 17078.5879
Epoch 231/10000
1/1 [==============================] - 0s 13ms/step - loss: 39713.2266 - val_loss: 17051.6309
Epoch 232/10000
1/1 [=====

1/1 [==============================] - 0s 13ms/step - loss: 34667.1445 - val_loss: 29938.5156
Epoch 298/10000
1/1 [==============================] - 0s 13ms/step - loss: 29981.5645 - val_loss: 26086.9590
Epoch 299/10000
1/1 [==============================] - 0s 13ms/step - loss: 47632.5312 - val_loss: 49701.7656
Epoch 300/10000
1/1 [==============================] - 0s 13ms/step - loss: 41842.8867 - val_loss: 36065.6445
Epoch 301/10000
1/1 [==============================] - 0s 13ms/step - loss: 40149.7500 - val_loss: 36544.4805
Epoch 302/10000
1/1 [==============================] - 0s 13ms/step - loss: 45928.6211 - val_loss: 53371.5469
Epoch 303/10000
1/1 [==============================] - 0s 13ms/step - loss: 29709.8379 - val_loss: 39784.4453
Epoch 304/10000
1/1 [==============================] - 0s 13ms/step - loss: 38959.4414 - val_loss: 42712.1367
Epoch 305/10000
1/1 [==============================] - 0s 13ms/step - loss: 26447.1348 - val_loss: 41721.6133
Epoch 306/10000
1/1 [=====

...

1/1 [==============================] - 0s 13ms/step - loss: 41782.6094 - val_loss: 74169.5391
Epoch 1185/10000
1/1 [==============================] - 0s 14ms/step - loss: 36337.4453 - val_loss: 73881.3750
Epoch 1186/10000
1/1 [==============================] - 0s 14ms/step - loss: 38900.8711 - val_loss: 76619.2188
Epoch 1187/10000
1/1 [==============================] - 0s 14ms/step - loss: 40374.7617 - val_loss: 79715.1484
Epoch 1188/10000
1/1 [==============================] - 0s 13ms/step - loss: 38825.3203 - val_loss: 74829.4922
Epoch 1189/10000
1/1 [==============================] - 0s 13ms/step - loss: 38720.3203 - val_loss: 75859.3047
Epoch 1190/10000
1/1 [==============================] - 0s 13ms/step - loss: 36490.3398 - val_loss: 72605.0078
Epoch 1191/10000
1/1 [==============================] - 0s 14ms/step - loss: 38653.6250 - val_loss: 76040.1406
Epoch 1192/10000
1/1 [==============================] - 0s 14ms/step - loss: 38572.1602 - val_loss: 74993.8672
Epoch 1193/10000
1

Epoch 1258/10000
1/1 [==============================] - 0s 13ms/step - loss: 24584.9414 - val_loss: 16212.2373
Epoch 1259/10000
1/1 [==============================] - 0s 13ms/step - loss: 27992.2383 - val_loss: 16701.9688
Epoch 1260/10000
1/1 [==============================] - 0s 14ms/step - loss: 13605.8682 - val_loss: 16091.4688
Epoch 1261/10000
1/1 [==============================] - 0s 13ms/step - loss: 24273.5020 - val_loss: 21548.3691
Epoch 1262/10000
1/1 [==============================] - 0s 13ms/step - loss: 22253.0781 - val_loss: 16185.7842
Epoch 1263/10000
1/1 [==============================] - 0s 13ms/step - loss: 14567.2725 - val_loss: 15974.1826
Epoch 1264/10000
1/1 [==============================] - 0s 13ms/step - loss: 21927.9648 - val_loss: 17408.1270
Epoch 1265/10000
1/1 [==============================] - 0s 13ms/step - loss: 15430.7188 - val_loss: 25641.1816
Epoch 1266/10000
1/1 [==============================] - 0s 13ms/step - loss: 29643.3301 - val_loss: 36531.1914
E

Epoch 1332/10000
1/1 [==============================] - 0s 13ms/step - loss: 40066.5508 - val_loss: 68675.1562
Epoch 1333/10000
1/1 [==============================] - 0s 13ms/step - loss: 40768.5469 - val_loss: 64814.4531
Epoch 1334/10000
1/1 [==============================] - 0s 13ms/step - loss: 24137.9043 - val_loss: 67983.3281
Epoch 1335/10000
1/1 [==============================] - 0s 13ms/step - loss: 33492.5078 - val_loss: 52081.6367
Epoch 1336/10000
1/1 [==============================] - 0s 13ms/step - loss: 37000.9492 - val_loss: 77388.9844
Epoch 1337/10000
1/1 [==============================] - 0s 13ms/step - loss: 34657.5273 - val_loss: 69397.8984
Epoch 1338/10000
1/1 [==============================] - 0s 13ms/step - loss: 31907.3379 - val_loss: 64855.7031
Epoch 1339/10000
1/1 [==============================] - 0s 13ms/step - loss: 30438.6348 - val_loss: 65852.7422
Epoch 1340/10000
1/1 [==============================] - 0s 13ms/step - loss: 28931.1445 - val_loss: 64570.4414
E

Epoch 1406/10000
1/1 [==============================] - 0s 13ms/step - loss: 32445.6504 - val_loss: 66070.3984
Epoch 1407/10000
1/1 [==============================] - 0s 13ms/step - loss: 27460.4023 - val_loss: 47937.0820
Epoch 1408/10000
1/1 [==============================] - 0s 13ms/step - loss: 20831.9551 - val_loss: 46606.4102
Epoch 1409/10000
1/1 [==============================] - 0s 13ms/step - loss: 24101.7188 - val_loss: 44954.1211
Epoch 1410/10000
1/1 [==============================] - 0s 13ms/step - loss: 28013.2344 - val_loss: 45981.6875
Epoch 1411/10000
1/1 [==============================] - 0s 13ms/step - loss: 38363.2930 - val_loss: 61649.0781
Epoch 1412/10000
1/1 [==============================] - 0s 13ms/step - loss: 33675.4805 - val_loss: 46464.5117
Epoch 1413/10000
1/1 [==============================] - 0s 13ms/step - loss: 38010.4727 - val_loss: 58732.9648
Epoch 1414/10000
1/1 [==============================] - 0s 13ms/step - loss: 41706.4805 - val_loss: 46624.7539
E

Epoch 1480/10000
1/1 [==============================] - 0s 13ms/step - loss: 33367.4688 - val_loss: 60495.8008
Epoch 1481/10000
1/1 [==============================] - 0s 13ms/step - loss: 36369.0234 - val_loss: 60575.8867
Epoch 1482/10000
1/1 [==============================] - 0s 13ms/step - loss: 32845.8047 - val_loss: 62672.2773
Epoch 1483/10000
1/1 [==============================] - 0s 13ms/step - loss: 31665.7715 - val_loss: 58744.2305
Epoch 1484/10000
1/1 [==============================] - 0s 13ms/step - loss: 32628.4551 - val_loss: 66952.5391
Epoch 1485/10000
1/1 [==============================] - 0s 13ms/step - loss: 33101.8711 - val_loss: 58247.6055
Epoch 1486/10000
1/1 [==============================] - 0s 13ms/step - loss: 35915.1836 - val_loss: 71798.2578
Epoch 1487/10000
1/1 [==============================] - 0s 13ms/step - loss: 31885.8633 - val_loss: 59923.2852
Epoch 1488/10000
1/1 [==============================] - 0s 13ms/step - loss: 26065.6328 - val_loss: 48329.3711
E

In [19]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 47ms/step
[1059.129](test_target) - [[1147.0154]](prediction) = 87.8863808593751


In [20]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [21]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - Desemprego,Maranhão - IDH,anhã-03o - Produção de Cimento (t),Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado
0,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,1.429992,1.039488,0.997366,...,-0.487834,-0.534391,0.720829,1.841595,-1.715002,-1.158615,-1.559807,-1.739742,-1.772310,-1.736663
1,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,0.375403,1.140589,1.088574,...,-0.651518,-1.121229,0.469895,1.021603,-1.108574,-0.789722,-1.059405,-1.290087,-1.115454,-1.078298
2,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-0.101810,1.128780,1.063523,...,-0.580414,-0.862692,0.994983,0.742268,-0.561962,-0.790688,-0.587124,-0.288593,-0.475454,-0.481570
3,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.718671,0.552952,0.614213,...,-0.684984,-0.598635,1.284930,-0.269150,-0.073882,-0.409716,-0.149665,1.030877,0.083272,0.035290
4,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,-1.157564,-0.513332,-0.243933,...,-0.615536,-0.003842,0.183009,-0.638280,0.356944,0.006330,0.216687,-0.165219,0.299453,0.298319
5,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,-1.331460,-0.958195,-0.905699,...,-0.143600,0.193222,-0.919137,-0.991426,0.731799,0.336310,0.579976,0.690588,0.800239,0.671216
6,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,0.029757,-1.184013,-1.218920,...,0.831320,0.745452,-1.394614,-0.635514,1.051962,0.631477,0.821891,0.650720,0.719399,0.686159
7,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,1.474353,-1.206269,-1.395124,...,2.332565,2.182115,-1.339895,-1.071096,1.318715,2.174624,1.737447,1.111456,1.460855,1.605547


train_target:


,Maranhão - Consumo de Cimento (t)
0,531.005
1,579.458
2,693.939
3,816.847
4,963.205
5,1123.586
6,1378.741
7,1282.016


test_input:


...

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - Desemprego,Maranhão - IDH,anhã-03o - Produção de Cimento (t),Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado
17,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,-0.547938,0.236151,-0.684249,...,0.061719,0.739102,-1.191,0.997695,1.039369,-0.622646,0.843673,-0.833746,0.580394,0.644008


test_target:


,Maranhão - Consumo de Cimento (t)
17,1059.129


1/1 [==============================] - 0s 34ms/step
Error: 87.8863808593751




[1188.8458251953125,
 1423.6617431640625,
 1461.589111328125,
 1309.3560791015625,
 1099.505859375,
 1290.69189453125,
 1693.6798095703125,
 1218.931396484375,
 959.7940063476562,
 1147.015380859375]

In [22]:
display(targets)
display(predictions)

[1528.77,
 1563.125,
 1303.4115714285715,
 1191.5096571428571,
 1100.2177714285715,
 971.921,
 750.012,
 774.045,
 1038.249,
 1059.129]

[1188.8458251953125,
 1423.6617431640625,
 1461.589111328125,
 1309.3560791015625,
 1099.505859375,
 1290.69189453125,
 1693.6798095703125,
 1218.931396484375,
 959.7940063476562,
 1147.015380859375]

In [23]:
mae = mean_absolute_error(predictions, targets)
mae

262.9789780650111

In [24]:
porcentage = mae/np.mean(targets)
porcentage

0.23312933157897123